
# DATA 101 – Session 2: Plotly Interactive Visualization (Comprehensive Demo)

**Goals**
- Use Plotly Express for quick, interactive charts.
- Use Plotly Graph Objects for fine-grained control.
- Add interactivity: hover, zoom, dropdowns, sliders, and animation.
- Export figures to HTML and static images.
- Consider performance and privacy.


In [ ]:

import sys, platform, importlib

def check_pkg(name):
    try:
        mod = importlib.import_module(name)
        return f"{name}=={getattr(mod, '__version__', 'unknown')}"
    except Exception as e:
        return f"{name} not available: {e}"

print("Python:", sys.version.split()[0], "|", platform.platform())
for pkg in ["plotly", "pandas", "numpy"]:
    print(check_pkg(pkg))

In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import kaleido
from palmerpenguins import load_penguins


In [ ]:
penguins = load_penguins()
penguins.head()

In [ ]:
gapminder = px.data.gapminder()
gapminder.head()


## Plotly Express Basics: Scatter

- High-level API with DataFrame-first design.
- Auto legends, color mapping, hover labels, axes.


In [ ]:

fig = px.scatter(
    penguins,
    x="bill_length_mm",
    y="bill_depth_mm",
    color="species",
    hover_name="island",
    title="Penguins: Bill Length vs Bill Depth"
)
fig.show()



## Bar charts and Faceting

- Quick categorical comparisons.
- Facet by column or row for small multiples.


In [ ]:

# Aggregate example: mean flipper length by species and island
agg = penguins.groupby(["island", "species"], as_index=False)["flipper_length_mm"].mean()

fig = px.bar(
    agg,
    x="species",
    y="flipper_length_mm",
    color="island",
    barmode="group",
    facet_col="island",
    title="Mean Flipper Length by Species and Island"
)
fig.update_layout(showlegend=False)
fig.show()



## Line charts with time series


In [ ]:

# Life expectancy over time for selected countries
countries = ["United States", "Japan", "Brazil", "Kenya"]
gm_sel = gapminder[gapminder["country"].isin(countries)]

fig = px.line(
    gm_sel,
    x="year",
    y="lifeExp",
    color="country",
    title="Life Expectancy Over Time"
)
fig.update_traces(mode="lines+markers")
fig.show()



## Interactivity: Dropdown menus

Toggle which country lines are visible using an **updatemenu**.


In [ ]:

countries = sorted(gm_sel["country"].unique())

# Build one trace per country for granular control
fig = go.Figure()
for c in countries:
    dfc = gm_sel[gm_sel["country"] == c]
    fig.add_trace(go.Scatter(x=dfc["year"], y=dfc["lifeExp"],
                             mode="lines+markers", name=c, visible=True))

buttons = []
for i, c in enumerate(countries):
    visible = [False]*len(countries)
    visible[i] = True
    buttons.append(dict(label=c, method="update", args=[{"visible": visible},
                    {"title": f"Life Expectancy: {c}"}]))

# Add an "All" option
buttons.insert(0, dict(label="All", method="update",
                       args=[{"visible": [True]*len(countries)},
                             {"title": "Life Expectancy Over Time (All)"}]))

fig.update_layout(
    updatemenus=[dict(type="dropdown", buttons=buttons, x=1.15, y=1.0)],
    title="Life Expectancy Over Time (Dropdown)",
    xaxis_title="Year",
    yaxis_title="Life Expectancy"
)
fig.show()



## Interactivity: Sliders and animation

Animate a bubble chart over time using `animation_frame`.


In [ ]:

fig = px.scatter(
    gapminder.query("year >= 1962"),
    x="gdpPercap", y="lifeExp",
    size="pop", color="continent",
    hover_name="country",
    log_x=True, size_max=45,
    animation_frame="year",
    range_x=[200, 70000],
    range_y=[30, 90],
    title="Gapminder: GDP vs Life Expectancy (Animated)"
)
fig.show()



## Exporting figures

- `fig.write_html("plot.html")` creates a standalone interactive HTML.
- `fig.write_image("plot.png")` creates a static image. Requires **kaleido**.


In [ ]:

# Reuse a simple figure for export demo
export_fig = px.scatter(penguins, x="bill_length_mm", y="bill_depth_mm",
                        color="species", title="Export Demo")
export_html = "./data/plot_export_demo.html"
export_fig.write_html(export_html)
print("Wrote HTML to:", export_html)

export_png = "./data/plot_export_demo.png"
export_fig.write_image(export_png, scale=2)
print("Wrote PNG to:", export_png)



## Plotly Graph Objects: fine control

- Add traces manually.
- Customize layout and annotations.
- Example: bar + line with a secondary y-axis.


In [ ]:

from plotly.subplots import make_subplots

np.random.seed(7)
x = np.arange(1, 13)
sales = np.random.randint(40, 120, size=12)
growth = np.round(np.random.uniform(-0.1, 0.2, size=12), 3)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=x, y=sales, name="Monthly Sales"), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=growth*100, name="Growth %", mode="lines+markers"),
              secondary_y=True)

fig.update_layout(title="Custom Combo Chart with Secondary Axis",
                  xaxis_title="Month",
                  legend_title="Series")
fig.update_yaxes(title_text="Sales (units)", secondary_y=False)
fig.update_yaxes(title_text="Growth (%)", secondary_y=True)

# Annotation example
fig.add_annotation(x=6, y=sales[5], text="Promo", showarrow=True, arrowhead=2)
fig.show()
